### Introduction

The goal of this notebook is to introduce autodE, a useful open-source software package that facilitate the construction of computational chemistry workflows, particularly those related to reactivity.

### autodE

Please start by installing -- and activating -- the conda environment present in the repository (conda env create -f environment.yml)

In [ ]:
import autode as ade
import os

os.chdir('work_dir') # to organize all the output-files

Below, the main functionality of the autodE package will be introduced step by step.

### Generate and manipulate atoms

In [ ]:
# Create a carbon atom
atom = ade.Atom("C")

# autodE atoms have a position
print("Position:  ", atom.coord)

# and useful properties, like the atomic number. See all of these properties
# here: https://duartegroup.github.io/autodE/reference/atoms.html
print("Z:         ", atom.atomic_number)

# and can be translated by a vector, for example 1 Å along the x axis
atom.translate([1.0, 0.0, 0.0])
print("Position:  ", atom.coord)

# or rotated
atom.rotate(axis=[0.0, 0.0, 1.0], theta=3.1415)  # z axis  # π radians
print("Position:  ", atom.coord)

# by default distances are in angstroms
atom.coord = [0.5, 0.0, 0.0]
print("Units are: ", atom.coord.units)

# and can be converted to others. For example, Bohr
coord_in_a0 = atom.coord.to("bohr")
print("Coordinate:", coord_in_a0, coord_in_a0.units)

# atoms can also be initialised at different positions
atom = ade.Atom("H", x=1.0, y=2.0, z=3.0)

# and their representation (repr) printed
print("H atom:    ", repr(atom))

### Generate chemical species and save their coordinates

In [ ]:
# Species in autodE are atom collections with a name,
# defined charge and spin multiplicity (mult). For example, to
# generate water from its three constituent atoms
h2o = ade.Species(
    name="water",
    charge=0,
    mult=1,
    atoms=[
        ade.Atom("O"),
        ade.Atom("H", x=-1.0),
        ade.Atom("H", x=0.21, y=-0.97),
    ],
)

# they have a number of properties, such as mass
print("Mass(H2O):       ", h2o.mass, h2o.mass.units)

# and the chemical formula
print("Formula:         ", h2o.formula)

# as well as radii *not including any van der Walls volume*
print("Approximate radius:", round(h2o.radius, 1), h2o.radius.units)

# with functions to calculate distances and angles between atoms
# For example, the distance between atoms 0 and 1:
print("O-H distance (Å):", h2o.distance(0, 1))
print("H-O-H angle (º): ", h2o.angle(1, 0, 2).to("degrees"))

# to save the structure and generate water.xyz in this directory
h2o.print_xyz_file()

Solvents can be included as follows:

In [ ]:
# Solvated species can be initialised with
h2 = ade.Species(
    name="h2_in_water",
    charge=0,
    mult=1,
    atoms=[ade.Atom("H"), ade.Atom("H", 0.77)],
    solvent_name="water",
)

print("H2 is solvated in:     ", h2.solvent)

# which are by default implicit solvated
print("Is solvated implicitly:", h2.is_implicitly_solvated)

# the associated solvent has properties, like ε
print("The dielectric constant is:     ", h2.solvent.dielectric)

# the solvent can be converted to explicit with -- THIS FEATURE IS NOT RECOMMENDED AS RESULTS ARE VERY SUSCEPTIBLE TO CONFORMATIONAL CHANGE
h2.explicitly_solvate(num=10)

print("Is solvated explicitly:", h2.is_explicitly_solvated)
print("Number of water atoms: ", h2.solvent.n_atoms)

# the whole solvated system can be printed
h2.print_xyz_file(filename="H2_solv.xyz")

### Generate, manipulate and optimize molecules

In [ ]:
# Molecules in autodE are just like species but can
# be initialised from SMILES strings. To generate methane
methane = ade.Molecule(smiles="C")

print(
    f"Methane has {methane.n_atoms} atoms, so \n"
    f"has a molecular graph with {methane.graph.number_of_nodes()}\n"
    f"nodes and {methane.graph.number_of_edges()} edges (bonds)."
)

# The whole molecule can be translated
methane.translate([1.0, 0.0, 0.0])
print("Translated carbon position is:", methane.coordinates[0, :])
# where the coordinates property is an Nx3 numpy array

# and rotated.
methane.rotate(axis=[0.0, 0.0, 1.0], theta=1.5)  # z axis  # radians
print("Rotated carbon position is:   ", methane.coordinates[0, :])

# Additionally calculations performed. To optimise the structure with XTB
xtb = ade.methods.XTB()
print(f"Using {ade.Config.n_cores} cores for an XTB calculation")

if xtb.is_available:
    methane.optimise(method=xtb)
    print("XTB energy (Ha):              ", methane.energy)

Advanced electronic structure programs are also available in autodE. A distinction is made between low- (xTB and MOPAC) and high-level codes (ORCA, Gaussian09, Gaussian16, NWChem, QChem). Many workflows will combine both levels of theory (e.g., conformer searches performed at low level of theory, followed by refined optimization of the lowest energy conformer with a high-level method). Because of time and resource constraints, we will not attempt to perform the high-level calculations here, but a worked out example is provided below.

In [ ]:
#ade.Config.hcode = 'g09'
g09 = ade.methods.G09()
ade.Config.lcode = 'xTB'

# To set the number of cores available and the memory per core (in MB), to use a maximum of 32 GB for the whole calculation
ade.Config.n_cores = 8
ade.Config.max_core = 4000

# autodE uses wrappers around common keywords used in QM calculations to allow easy setting of e.g. a DFT functional
kwds = g09.keywords.sp
print(f'The selected functional is: {kwds.functional}')

# Alternatively, a whole new set of keywords can be assigned:
g09.keywords.sp = ['SP', 'B3LYP', 'def2-TZVP']
print(f'The selected keywords are: {kwds.functional}')

# TODO: Fix this!!!!
print(f"Using {ade.Config.n_cores} cores for a Gaussian09 calculation")

if g09.is_available:
    print(f"Calculating at the: [{g09.keywords.sp}] level of theory")
    methane.single_point(method=g09)
    print("G09 energy (Ha):             ", methane.energy)
else:
    print("G09 method is not available; skipping the single-point energy refinement")

# with all energies available -- in this case only the xTB one
print("All calculated energies:      ", methane.energies)

Molecules can also be initialized from xyz-files

In [ ]:
# Molecules can be initialised directly from 3D structures
serine = ade.Molecule("../data/serine.xyz")

# molecules initialised from .xyz files default to neutral singlets
print("Name:                    ", serine.name)
print("Charge:                  ", serine.charge)
print("Spin multiplicity:       ", serine.mult)
print("Is solvated?:            ", serine.solvent is not None)

# dihedrals can also be evaluated evaluated
symbols = "-".join(serine.atomic_symbols[:4])
print(f"{symbols} dihedral:        ", serine.dihedral(0, 1, 2, 3), "radians")

# an estimated molecular graph is initialised.
# NOTE: This will be less accurate for organometallic species
print("Bond matrix for the first 4 atoms:\n", serine.bond_matrix[:4, :4])

# molecules also have a has_same_connectivity_as method, which
# checks if the molecular graph is isomorphic to another
blank_mol = ade.Molecule()
print("Num atoms in a empty mol:", blank_mol.n_atoms)
print(
    "Graph is isomorphic to an empty graph:           ",
    serine.has_same_connectivity_as(blank_mol),
)

In [ ]:
# Create a serine molecule from a SMILES string
serine_from_smiles = ade.Molecule(smiles="N[C@@H](CO)C(O)=O")
print(
    "Graph is isomorphic to SMILES-generated molecule:",
    serine.has_same_connectivity_as(serine_from_smiles),
)

### Conformer generation

autodE generates conformers using two methods: (1) ETKDGv3 implemented in RDKit and (2) a randomize & relax (RR) algorithm. For regular organic compounds, the former method is used by default, for metal complexes, the latter is used.

In [ ]:
# Conformers of organic molecules initalised from SMILES strings
# in autodE are generated using RDKit. For example,
pentane = ade.Molecule(name='butane', smiles="CCCCC")

print("Num. initial conformers:    ", pentane.n_conformers)
print("Initial C-C distance (Å):   ", pentane.distance(0, 1))

# To generate a set of conformers
pentane.populate_conformers(n_confs=10)

print("Num. generated conformers:  ", pentane.n_conformers)
# NOTE: the number of generated conformers is usually smaller than
# the number requested, as they are pruned based on similarity
value = ade.Config.rmsd_threshold
print("Default pruning tolerance:  ", value, value.units)

# To find the lowest energy conformer by optimising at XTB then
# re-optimising the unique ones at a higher level
xtb = ade.methods.XTB()

pentane.find_lowest_energy_conformer(lmethod=xtb)

# find_lowest_energy_conformer will set the molecule's geometry and energy
print("Optimised C-C distance (Å): ", pentane.distance(0, 1))
print("Potential energy:           ", pentane.energy, pentane.energy.units)
print("Pruned number of conformers:", pentane.n_conformers)

Arbitrary distance constraints can be added in a RR conformer generation. For example, to generate conformers of Vaska’s complex while retaining the square planar geometry:

In [ ]:
from autode.conformers import conf_gen, Conformer

# Initialise the complex from a .xyz file containing a square planar structure
vaskas = ade.Molecule("../data/vaskas.xyz")

# Set up some distance constraints where the keys are the atom indexes and
# the value the distance in Å. Fixing the Cl-P, Cl-P and Cl-C(=O) distances
# enforces a square planar geometry
distance_constraints = {
    (1, 2): vaskas.distance(1, 2),
    (1, 3): vaskas.distance(1, 3),
    (1, 4): vaskas.distance(1, 4),
}

# Generate 5 conformers
for n in range(5):
    # Apply random displacements to each atom and minimise under a bonded +
    # repulsive forcefield including the distance constraints
    atoms = conf_gen.get_simanl_atoms(
        species=vaskas, dist_consts=distance_constraints, conf_n=n
    )

    # Generate a conformer from these atoms then optimise with XTB
    conformer = Conformer(name=f"vaskas_conf{n}", atoms=atoms)

    conformer.optimise(method=ade.methods.XTB())
    conformer.print_xyz_file()

### Constrained optimization 

In [ ]:
xtb = ade.methods.XTB()

# Constrained optimisations are possible by setting a molecule's constraints
# attribute, for example to calculate the relaxed PES for H-transfer from
# the neutral form of serine to the zwitterion

serine = ade.Molecule("../data/serine.xyz", solvent_name="water")

print("Current N-H distance (Å):", serine.distance(0, 13))

energies = []
for r in (2.4, 2.2, 2.0, 1.8, 1.6, 1.4, 1.2, 1.0):
    # Set the distance constraint between atoms 0 and 13
    serine.constraints.distance = {(0, 13): r}

    # optimise with XTB
    serine.optimise(method=xtb)

    # and append the energies to a list
    energies.append(serine.energy)

print("Final N-H distance is:  ", serine.distance(0, 13))
print("Energies along the path:", energies)

In [ ]:
# Cartesian coordinates can also be fixed. For example, to optimise BH3
# while keeping two H atoms 2 Å apart

bh3 = ade.Molecule(
    atoms=[
        ade.Atom("B", y=0.1),
        ade.Atom("H", x=-1.0),
        ade.Atom("H", x=1.0),
        ade.Atom("H", y=1.1),
    ]
)

print("Current H-B-H angle (º):  ", bh3.angle(1, 0, 2).to("º"))

# Set the constraints and do the optimisation
bh3.constraints.cartesian = [1, 2]
bh3.optimise(method=xtb)

print("Optimised H-B-H angle (º):", bh3.angle(1, 0, 2).to("º"))

### Thermochemistry

In [ ]:
# Create and optimise an ammonia molecule
nh3 = ade.Molecule(smiles="N")
nh3.optimise(method=xtb)

# Calculate the thermochemistry by running a Hessian calculation at the
# default level of theory
nh3.calc_thermo(method=xtb)

print("Zero-point energy               =", nh3.zpe.to("kJ mol-1"), "kJ mol-1")
print("Enthalpy contribution           =", nh3.h_cont)
print("Free energy contribution        =", nh3.g_cont)
print("Total free energy               =", nh3.free_energy)

print("Frequencies:", [freq.to("cm-1") for freq in nh3.vib_frequencies])

# Frequencies have a is_imaginary property. To print the number of imaginary-s:
print(
    "Number of imaginary frequencies:",
    sum(freq.is_imaginary for freq in nh3.vib_frequencies),
)

### Normal modes

In [ ]:
# Optimise and calculate the Hessian for a water molecule
h2o = ade.Molecule(smiles="O")
h2o.optimise(method=xtb)
h2o.calc_hessian(method=xtb)

print("Number of total frequencies is:      ", 3 * h2o.n_atoms)
print("Number of vibrational frequencies is:", len(h2o.vib_frequencies))
print(
    "Frequencies in wave-numbers:         ",
    [float(nu) for nu in h2o.vib_frequencies],
)

# Now, generate a set of normal mode-displaced h2o molecules displaced along
# the highest frequency normal mode (index 8), where 0-2 are translations
# 3-5 are rotations and (6, 7, 8) are vibrations
mode = h2o.normal_mode(8)

for i in range(30):
    h2o.coordinates += 0.01 * mode
    h2o.print_xyz_file(filename="h2o_mode2.xyz", append=True)

### Hessians

In [ ]:
# Dinitrogen molecule
n2 = ade.Molecule(smiles="N#N")

n2.optimise(method=xtb)
n2.calc_hessian(
    method=xtb, numerical=True, use_central_differences=True
)

print(f"Numerical frequency calculated at {xtb.name} level-of-theory:", n2.vib_frequencies)

### Generate potential energy surface plots

In [ ]:
# 2+ dimensional PESs can also be calculated. For example, considering the
# identity reaction CH3 + CH4 -> CH4 + CH3
reactive_complex = ade.Molecule("../data/CH3_CH4.xyz", mult=2)

# Create then calculate the PES
pes = ade.pes.RelaxedPESnD(
    reactive_complex,
    rs={(0, 1): (3.0, 8), (5, 1): (1.1, 8)},  # Current->3.0 Å in 8 steps
)  # Current->1.1 Å in 8 steps
pes.calculate(
    method=xtb
)  # Using 10 processing cores

# and plot the 2D surface
pes.plot(filename="CH3_CH4.pdf")

### Reaction profiles

As indicated before, we will not be using the interface of advanced electronic structure codes in this tutorial. However, as xTB does not include methods to search for transition states, one cannot generate reaction profiles exclusively with these methods. Since reaction profile generation is one of the key features of autodE, we include below a coding example to run such a calculation -- this cell will currently not work and is purely included as illustration; the output it generates can be inspected in the data folder ('data/reaction').

In [ ]:
ade.Config.lcode = "xtb"
ade.Config.hcode = "g09"

# Full reaction profiles can be calculated by again forming a reaction
# and calling calculate_reaction_profile. Conformers will be searched,
# a TS found and single point energies evaluated. The reaction is defined a
# as a single string, with reactants and products separated by '>>'
rxn = ade.Reaction("CCl.[F-]>>CF.[Cl-]", solvent_name="water")

print(f"Calculating the reaction profile for {rxn.reacs}->{rxn.prods}")

try:
    rxn.calculate_reaction_profile()
    print("∆E‡ =", rxn.delta("E‡").to("kcal mol-1"))
except:
    print('Calculation failed because high method not available')

With the main functionality of autodE to explore chemical reactivity in an automated manner explained, you can now proceed to a second Notebook, namely: combinatorial_dataset_construction.ipynb. Alternatively, you can take at the intro_ts_tools.ipynb Notebook, to learn about another open-source software tool for mechanistic elucidation.